# Coronavirus distribution maps in MX 
Source: Government open coronavirus database, INEGI coordenates catalog

In [2]:
###### ------------------- Main libraries -------------------################
import pandas as pd
import numpy as np 
from pprint import pprint
import folium 
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import datetime

# Coordenates Catalog 

In [4]:
coordenadas = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/coordenadas.csv')
coordenadas.head()

C:\Users\Data Analyst\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Mapa,Cve_Ent,Nom_Ent,Nom_Abr,Cve_Mun,Nom_Mun,Cve_Loc,Nom_Loc,Ãmbito,Latitud,Longitud,Lat_Decimal,Lon_Decimal,Altitud,Cve_Carta,Pob_Total,Pob_Masculina,Pob_Femenina,Total De Viviendas Habitadas
0,10010001,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21Â°52Â´47.362N""","102Â°17Â´45.768W""",21.879823,-102.296047,1878,F13D19,722250,348722,373528,185120
1,10010094,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21Â°52Â´18.749N""","102Â°22Â´24.710W""",21.871875,-102.373531,1902,F13D18,14,*,*,2
2,10010096,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21Â°53Â´01.522N""","102Â°21Â´25.639W""",21.883756,-102.357122,1861,F13D18,37,21,16,11
3,10010100,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21Â°51Â´16.556N""","102Â°22Â´21.884W""",21.854599,-102.372746,1879,F13D18,10,*,*,1
4,10010102,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21Â°46Â´48.650N""","102Â°21Â´26.261W""",21.780181,-102.357295,1861,F13D18,7,*,*,1


# MCP Catalog

In [5]:
municipios = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/catalogo_municipios.csv')
municipios.head()

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD
0,1,AGUASCALIENTES,1
1,2,ASIENTOS,1
2,3,CALVILLO,1
3,4,COSÍO,1
4,5,JESÚS MARÍA,1
